In [3]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import sys
import json

In [4]:
from TextItem import TextItem

In [10]:
save_folder = "STEP 1 - RAW_JSON_TO_CSV"
path = os.path.join(os.path.dirname(os.getcwd()), "graphql", "EVALUATIONS")
path

'C:\\Users\\Tollef\\Documents\\GitHub\\masterNEW\\REPO\\graphql\\EVALUATIONS'

In [11]:
def get_if_exists(object, key):
    if key in object:
        return object[key]
    return None

In [12]:
entity_lookup_dictionary = set()

print("Building lookup dictionary")
for folder in os.listdir(path):
    sub_path = os.path.join(path, folder)
    print(sub_path)
    topic_dfs = []
    for jsonfile in os.listdir(sub_path):
        jsonfile = os.path.join(sub_path, jsonfile)
        print(jsonfile)
        with open(jsonfile, 'r', encoding="utf8") as f:
            f = json.load(f)
            edges = f["events"]["edges"]
            for node in edges:
                subnode = node["node"]
                entities = get_if_exists(subnode, "entities")
                entity_edges = get_if_exists(entities, "edges")
                
                for entity_object in entity_edges:
                    entity = get_if_exists(entity_object, "node")
                    if not entity:
                        continue

                    entity_name = get_if_exists(entity, "name")
                    if not entity_name:
                        continue
                    
                    entity_lookup_dictionary.add(entity_name)

Building lookup dictionary
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\business
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\business\2020_Q1_3.json
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\politics
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\politics\2020_Q1_3.json
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\sports
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\sports\2020_Q1_3.json
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\tech
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\tech\2020_Q1_3.json


In [13]:
len(entity_lookup_dictionary)

7745

In [14]:
final_entity_dfs = []
final_texts_dfs = []

for folder in os.listdir(path):
    sub_path = os.path.join(path, folder)
    print(sub_path)
    topic_dfs = []
    for jsonfile in os.listdir(sub_path):
        jsonfile = os.path.join(sub_path, jsonfile)
        print(jsonfile)
        with open(jsonfile, 'r', encoding="utf8") as f:
            f = json.load(f)
            edges = f["events"]["edges"]
            print("Edges count:", len(edges))
            
            entities = []
            texts = []
            
            for node in edges:
                data = TextItem(node, entity_lookup_dictionary, limit_to_relations=True)
                ents = data.get_entities_object()
                txts = data.get_text_object()
                
                entities.append(pd.DataFrame(ents))
                texts.append(txts)
                
            
            entity_df = pd.concat(entities)            
            text_df = pd.DataFrame(texts)
                        
            topic_dfs.append(entity_df)
            
            final_entity_dfs.append(entity_df)
            final_texts_dfs.append(text_df)    
            
    # store the entity_df as topic-oriented entities separately
    df_name = "{}_{}".format(folder, "entities.csv")
    df_path = os.path.join(os.getcwd(), save_folder, df_name)
    print("Storing topic DF at\n", df_path)

    topic_df = pd.concat(topic_dfs)
    topic_df.to_csv(df_path)
FINAL_ENTITY_DF = pd.concat(final_entity_dfs)
FINAL_TEXTS_DF = pd.concat(final_texts_dfs)

# set default values for sentiment and discard (used for discarding during manual labeling)
# FINAL_ENTITY_DF['sentiment'] = 0
# FINAL_ENTITY_DF['to_discard'] = False

C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\business
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\business\2020_Q1_3.json
Edges count: 100
Storing topic DF at
 C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 1 - RAW_JSON_TO_CSV\business_entities.csv
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\politics
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\politics\2020_Q1_3.json
Edges count: 100
Storing topic DF at
 C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 1 - RAW_JSON_TO_CSV\politics_entities.csv
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\sports
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\sports\2020_Q1_3.json
Edges count: 100
Storing topic DF at
 C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\strise\STEP 1 - RAW_JSON_TO_CSV\sports_entities.csv
C:\Users\Tollef\Documents\GitHub\masterNEW\REPO\graphql\EVALUATIONS\tech
C:\Use

In [16]:
FINAL_ENTITY_DF.to_csv(save_folder + "/entities.csv", index=False)
FINAL_TEXTS_DF.to_csv(save_folder + "/texts.csv", index=False)